Version notes (f)
- split out by market and tailor dtd clipping

In [1]:
import datetime

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1669997821946_0002,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# this data has been updated to include holiday indicator
last_shop_day = "20221130"
df = spark.read.parquet(f"s3://tvlp-ds-users/kendra-frederick/shopping-grid/engineered-features_v3/parquet/{last_shop_day}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- market: string (nullable = true)
 |-- out_departure_date: long (nullable = true)
 |-- shop_date: long (nullable = true)
 |-- fare_rank: integer (nullable = true)
 |-- num_itin: long (nullable = true)
 |-- fare: double (nullable = true)
 |-- out_cxrs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- solution_counts: long (nullable = true)
 |-- shop_counts: long (nullable = true)
 |-- avg_out_avail_max: double (nullable = true)
 |-- avg_out_avail_low: double (nullable = true)
 |-- outDeptDt_dt: date (nullable = true)
 |-- shopDate_dt: date (nullable = true)
 |-- days_til_dept: integer (nullable = true)
 |-- dept_dt_dow_int: integer (nullable = true)
 |-- dept_dt_dow: string (nullable = true)
 |-- is_holiday: integer (nullable = true)
 |-- fr2_fare: double (nullable = true)
 |-- fr2_out_cxrs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fr2_avg_out_avail_max: double (nullable = true)
 |-- fr2_avg_out_avail_low: double (

# define features

In [4]:
target_col = 'fare'

# scale: mean = 0, std = 1
features_to_scale = [
    # fare-based features
    'fare_prev_shop_day',
    'fr2_fare_prev_shop_day', 
    'avg_fare_dtd',
    'fare_prev_dept_day', 'fare_next_dept_day',
    'est_fare_from_prev_dept_day', 'est_fare_from_next_dept_day',
    'trailing_avg_fare', 'trailing_std_fare',
                    ]
# norm: [0,1]
features_to_norm = [
    'days_til_dept', # encoded in 'avg_fare_dtd', so exclude?
    'num_itin_prev_shop_day',
    # count-based features - expect some skew in these, so norm instead of scale (?)
    'shop_counts_prev_shop_day', 
    'trailing_avg_shop_counts',
    'solution_counts_prev_shop_day',
    'trailing_avg_solution_counts',
]
categorial_features = [
    'dept_dt_dow_int' 
]
as_is_features = [
    # avail-based features are already norm'd
    'avg_out_avail_max_prev_shop_day',
    'avg_out_avail_low_prev_shop_day',
    # boolean feature(s) - new in version d
    'fr1_fr2_out_cxrs_same',
    'fr1_fr2_out_cxrs_overlap',
    'is_holiday'
]
all_features = features_to_scale + features_to_norm + categorial_features + as_is_features
all_cols_for_modeling = all_features + [target_col]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# prep data

## clip

In [5]:
df.select(F.min("shopDate_dt")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|min(shopDate_dt)|
+----------------+
|      2022-10-29|
+----------------+

based on shop date (given that we took a 3-day trailing average -- the first 2 days won't have valid data)

In [6]:
first_shop_date = datetime.date(2022,10,29)
window_size = 3
shop_cutoff = first_shop_date + datetime.timedelta(days=window_size)
df_clip_shop = df.filter(F.col('shopDate_dt') >= shop_cutoff)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

By days til dept -- such as 120

In [7]:
dtd_grp = df.groupby('days_til_dept').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
dtd_grp.orderBy("days_til_dept").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|days_til_dept|count|
+-------------+-----+
|            4|  448|
|            5|  448|
|            6|  448|
|            7|  448|
|            8|  448|
+-------------+-----+
only showing top 5 rows

Already clipped off the first 3

In [10]:
dtd_limit = 120
df_clip_dtd = df_clip_shop.filter(F.col("days_til_dept") <= 120)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.count(), df_clip_shop.count(), df_clip_dtd.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(133361, 120659, 47501)

This cuts out a lot of data, but so be it

## dealing with nulls

Drop for now. Revisit later

In [12]:
df_no_nulls = df_clip_dtd.dropna(subset=all_cols_for_modeling)
df_no_nulls.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

47092

## Train-test split

In [13]:
train, test = df_no_nulls.randomSplit([0.8, 0.2], seed=19)
train_shuffled = train.orderBy(F.rand())

df_no_holidays = df_no_nulls.filter(F.col("is_holiday") == 0)
train_no_hol, test_no_hol = df_no_holidays.randomSplit([0.8, 0.2], seed=19)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# modeling prep

## import things

In [14]:
from pyspark.ml.feature import (
    MinMaxScaler, StandardScaler, OneHotEncoderEstimator,
    VectorAssembler, VectorIndexer
)
from pyspark.ml.regression import (
    LinearRegression, RandomForestRegressor, GBTRegressor
)
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## eval code

In [15]:
def calc_abs_pct_err(df, pred_col="prediction"):
    df = (df
           .withColumn("err", F.col("fare") - F.col(pred_col))
           .withColumn("abs_err", F.abs(F.col("err")))
           .withColumn("abs_pct_err", F.col("abs_err") / F.col("fare"))
          )
    print(df.select(F.mean("abs_pct_err").alias("mape")).show())
    # err_by_market = df.groupBy("market").agg(F.mean("abs_pct_err").alias("mape")).orderBy("mape")
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Define Pipelines & quick fit-check

Useful refs:
- https://spark.apache.org/docs/2.4.8/api/python/pyspark.ml.html
    - For a specific algo, see `setParams` method. This enumerates the parameters accepted.

## Random Forest

Best params from tuning were: 100 tree, maxDepth 7

In [18]:
raw_feature_assembler = VectorAssembler(
    inputCols=all_features,
    outputCol='features'
)

rf_regression_model = RandomForestRegressor(
    labelCol="fare",
    numTrees=100,
    maxDepth=7,
)

rf_pipeline = Pipeline(stages=[
    raw_feature_assembler,
    rf_regression_model
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
rf_model = rf_pipeline.fit(train_shuffled)
pred_rf_df = rf_model.transform(test)
pred_rf_df = calc_abs_pct_err(pred_rf_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|               mape|
+-------------------+
|0.06425197397085082|
+-------------------+

None

In [22]:
rf_model = rf_pipeline.fit(train_no_hol)
pred_rf_df = rf_model.transform(test_no_hol)
pred_rf_df = calc_abs_pct_err(pred_rf_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|              mape|
+------------------+
|0.0549286203820055|
+------------------+

None

In the range of what we saw before

## Linear Regression

In [20]:
ohe = OneHotEncoderEstimator(
    inputCols=categorial_features,
    outputCols=['dept_dow_enc']
)

scale_assembler = VectorAssembler(
    inputCols=features_to_scale,
    outputCol="features_to_scale"
)

scaler = StandardScaler(
    inputCol=scale_assembler.getOutputCol(),
    outputCol="scaled_features"
)

norm_assembler = VectorAssembler(
    inputCols=features_to_norm,
    outputCol="features_to_norm"
)

normer = MinMaxScaler(
    inputCol=norm_assembler.getOutputCol(),
    outputCol="normed_features"
)

# Note: if shuffling data doesn't work, add a VectorSizeHint here
# https://spark.apache.org/docs/2.4.8/ml-features.html#vectorsizehint
# I think this is what's going on: https://www.matthewslipper.com/2017/10/01/watch-out-vectorassembler-incompatible-spark-streaming.html
final_assembler = VectorAssembler(
    inputCols=[*ohe.getOutputCols(), 
               scaler.getOutputCol(),
               normer.getOutputCol()
              ] + as_is_features,
    outputCol='features'
)

lin_regression_model = LinearRegression(
    #  elasticNetParam corresponds to α (ratio) and regParam corresponds to λ (weight).
    labelCol="fare",
    maxIter=20,
    regParam=1.0,
    elasticNetParam=0.01
)

lin_pipeline = Pipeline(stages=[
    scale_assembler,
    norm_assembler,
    ohe, 
    scaler,
    normer,
    final_assembler,
    lin_regression_model
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
lr_model = lin_pipeline.fit(train_shuffled)
pred_lr_df = lr_model.transform(test)
pred_lr_df = calc_abs_pct_err(pred_lr_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|               mape|
+-------------------+
|0.08362963197983488|
+-------------------+

None

In [23]:
lr_model = lin_pipeline.fit(train_no_hol)
pred_lr_df = lr_model.transform(test_no_hol)
pred_lr_df = calc_abs_pct_err(pred_lr_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|               mape|
+-------------------+
|0.06638198343686914|
+-------------------+

None

## Gradient Boosing

In [33]:
raw_feature_assembler = VectorAssembler(
    inputCols=all_features,
    outputCol='features'
)

gb_regression_model = GBTRegressor(
    labelCol="fare",
    # maxIter=20, # this is default
    # maxDepth=5, # this is default
    stepSize=0.1,
)

gb_pipeline = Pipeline(stages=[
    raw_feature_assembler,
    gb_regression_model
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-33:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



GBT messes up the output, so do it last

In [34]:
gb_model = gb_pipeline.fit(train_no_hol)
pred_gb_df = gb_model.transform(test_no_hol)
pred_gb_df = calc_abs_pct_err(pred_gb_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-34:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.06205723630216071|
+-------------------+

None

# Tune

In [26]:
def get_cv_results_list(cv_model, print_vals=True):
    cv_metrics = cv_model.avgMetrics
    cv_params = [dict(zip([y.name for y in x.keys()], x.values())) for x in list(cv_model.getEstimatorParamMaps())]
    cv_results = list(zip(cv_params, cv_metrics))
    if print_vals:
        for x in cv_results:
            print(x[0], x[1])
    return cv_results


def get_best_from_cv(cv_model):
    cv_results = list(zip(
        cv_model.getEstimatorParamMaps(), 
        cv_model.avgMetrics
    ))
    cv_results.sort(key=lambda x: x[1])
    best_result = cv_results[0]
    best_metric = best_result[1]
    best_param_dict = best_result[0]
    params = [x.name for x in list(best_param_dict.keys())]
    param_vals = best_param_dict.values()
    best_params = dict(zip(params, param_vals))
    print(best_params, best_metric)
    return best_params

VBox()

Exception in thread cell_monitor-26:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## RFR

In [25]:
rf_param_grid = (ParamGridBuilder()
              .addGrid(rf_regression_model.numTrees, [50, 100, 200])
              .addGrid(rf_regression_model.maxDepth, [3, 5, 7, 10])
              .build()
             )

rf_cv = CrossValidator(estimator=rf_pipeline,
                    estimatorParamMaps=rf_param_grid,
                    evaluator=RegressionEvaluator(labelCol="fare"),
                    numFolds=3,
                    parallelism=5
                   )

rf_cv_model = rf_cv.fit(train_no_hol)
pred_rf_cv = rf_cv_model.transform(test_no_hol)
pred_rf_cv = calc_abs_pct_err(pred_rf_cv)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-25:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.04640951809656454|
+-------------------+

None

Same-ish as before

In [27]:
get_best_from_cv(rf_cv_model)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'numTrees': 200, 'maxDepth': 10} 19.543118876363895
{'numTrees': 200, 'maxDepth': 10}

Exception in thread cell_monitor-27:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [28]:
get_cv_results_list(rf_cv_model)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'numTrees': 50, 'maxDepth': 3} 28.943790962496827
{'numTrees': 50, 'maxDepth': 5} 21.696493694630657
{'numTrees': 50, 'maxDepth': 7} 20.409910452933378
{'numTrees': 50, 'maxDepth': 10} 19.59891077879365
{'numTrees': 100, 'maxDepth': 3} 28.846583012696193
{'numTrees': 100, 'maxDepth': 5} 21.673347775403652
{'numTrees': 100, 'maxDepth': 7} 20.368780921639683
{'numTrees': 100, 'maxDepth': 10} 19.593892974023653
{'numTrees': 200, 'maxDepth': 3} 28.759434386415244
{'numTrees': 200, 'maxDepth': 5} 21.708060290574455
{'numTrees': 200, 'maxDepth': 7} 20.353285735004757
{'numTrees': 200, 'maxDepth': 10} 19.543118876363895
[({'numTrees': 50, 'maxDepth': 3}, 28.943790962496827), ({'numTrees': 50, 'maxDepth': 5}, 21.696493694630657), ({'numTrees': 50, 'maxDepth': 7}, 20.409910452933378), ({'numTrees': 50, 'maxDepth': 10}, 19.59891077879365), ({'numTrees': 100, 'maxDepth': 3}, 28.846583012696193), ({'numTrees': 100, 'maxDepth': 5}, 21.673347775403652), ({'numTrees': 100, 'maxDepth': 7}, 20.3687809

Exception in thread cell_monitor-28:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



Looks like maxDepth is more important than numTrees -- so could set numTrees to 50 or 100 and be OK

In [45]:
pred_rf_cv.groupBy("market").agg(F.mean("abs_pct_err").alias("mape")).orderBy("mape").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-45:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------+--------------------+
| market|                mape|
+-------+--------------------+
|EWR-LHR|0.015811609564935016|
|LHR-JFK|0.018773058458982576|
|LHR-EWR|0.019062102237868986|
|EWR-CDG|0.021300113325203648|
|LAX-SFO| 0.02846503624735155|
|SFO-LAX|0.033914739850896505|
|JFK-LHR|0.035179532982902655|
|LHR-LAX|0.042434450467471484|
|LAX-JFK| 0.04459732349711695|
|JFK-LAX| 0.04868007486205408|
|ATL-EWR|0.052720328846765266|
|LAX-EWR| 0.06034662651003775|
|LGA-MIA|   0.082259064770822|
|OAK-LAS| 0.14206486427769252|
+-------+--------------------+

## LR

In [29]:
lr_param_grid = (ParamGridBuilder()
              # regParam is lambda (weight / factor), elasticNet is alpha (ratio)
              # note alpha is the opposite of sklearn's defintion
              .addGrid(lin_regression_model.elasticNetParam, [0.01, 0.1, 0.5, 0.9, 0.99])
              .addGrid(lin_regression_model.regParam, [0.01, 0.1, 1, 10, 100])
              .build()
             )

lr_cv = CrossValidator(estimator=lin_pipeline,
                    estimatorParamMaps=lr_param_grid,
                    evaluator=RegressionEvaluator(labelCol="fare"),
                    numFolds=3,
                    parallelism=5
                   )

lr_cv_model = lr_cv.fit(train_no_hol)
pred_lr_cv = lr_cv_model.transform(test_no_hol)
pred_lr_cv = calc_abs_pct_err(pred_lr_cv)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-29:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+------------------+
|              mape|
+------------------+
|0.0702866120448544|
+------------------+

None

Actually worse than above

In [31]:
get_best_from_cv(lr_cv_model)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'elasticNetParam': 0.01, 'regParam': 0.1} 19.921439570672902
{'elasticNetParam': 0.01, 'regParam': 0.1}

Exception in thread cell_monitor-31:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [32]:
get_cv_results_list(lr_cv_model)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'elasticNetParam': 0.01, 'regParam': 0.01} 20.33374745615661
{'elasticNetParam': 0.01, 'regParam': 0.1} 19.921439570672902
{'elasticNetParam': 0.01, 'regParam': 1.0} 20.106381844114868
{'elasticNetParam': 0.01, 'regParam': 10.0} 21.223565888160948
{'elasticNetParam': 0.01, 'regParam': 100.0} 26.15946292202313
{'elasticNetParam': 0.1, 'regParam': 0.01} 19.929083900557327
{'elasticNetParam': 0.1, 'regParam': 0.1} 20.43450844156545
{'elasticNetParam': 0.1, 'regParam': 1.0} 20.361612158286427
{'elasticNetParam': 0.1, 'regParam': 10.0} 21.14601936556225
{'elasticNetParam': 0.1, 'regParam': 100.0} 30.21174862056361
{'elasticNetParam': 0.5, 'regParam': 0.01} 19.927766074112903
{'elasticNetParam': 0.5, 'regParam': 0.1} 20.23516065162236
{'elasticNetParam': 0.5, 'regParam': 1.0} 20.030397193777635
{'elasticNetParam': 0.5, 'regParam': 10.0} 21.24320146073316
{'elasticNetParam': 0.5, 'regParam': 100.0} 59.34775957401261
{'elasticNetParam': 0.9, 'regParam': 0.01} 20.466928790602815
{'elasticNetPa

Exception in thread cell_monitor-32:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



Most ~19-20, except for a few

# feature importances

In [41]:
import matplotlib.pyplot as plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-41:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [37]:
def get_feature_importances(pipeline_model, is_cross_val_model, print_imps=True):
    """
    pipeline_model: fitted pipeline. Assumes last stage is an ML model
        that has `.featureImportances` attribue, and penultimate stage 
        is a VectorAssembler whose `inputCols` are feature names
    is_cross_val_model (bool): whether `pipeline_model` is a 
        CrossValidatorModel
    print (bool): whether to print out the sorted list of feature
        importances
    """
    # FEATURE IMPORTANCE
    if is_cross_val_model:
        best_pipeline = pipeline_model.bestModel
        # va = vector assembler; assumed to be stage just before ML algo,
        # which has feature names
        va = best_pipeline.stages[-2]
        algo = best_pipeline.stages[-1]
    else:
        va = pipeline_model.stages[-2]
        algo = pipeline_model.stages[-1]

    feat_imps = list(zip(va.getInputCols(), algo.featureImportances))


    feat_imps.sort(key=lambda x: x[1], reverse=True)
    if print_imps:
        for x in feat_imps:
            print(x)
    return feat_imps

def plot_feature_impts(feat_imps, file_name, plot_title):
    """
    feat_imps: list of tuples of (feature name, feature importance)
    """
    xs = range(len(feat_imps))
    plt.figure(figsize=(8,6))
    plt.bar(xs, [x[1] for x in feat_imps])
    plt.xticks(xs, [x[0] for x in feat_imps], rotation=90)
    plt.ylabel("feature importance")
    plt.title(plot_title)
    plt.tight_layout()
    plt.savefig(f"/tmp/{file_name}.png")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-37:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [38]:
rf_feat_imps = get_feature_importances(rf_cv_model, True, print_imps=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('fare_prev_shop_day', 0.3113814012233963)
('trailing_avg_fare', 0.25049541602749226)
('fr2_fare_prev_shop_day', 0.1485442479832904)
('est_fare_from_next_dept_day', 0.09075394027178695)
('est_fare_from_prev_dept_day', 0.06855169966705582)
('fare_prev_dept_day', 0.06287087219811671)
('fare_next_dept_day', 0.04086388909387464)
('avg_fare_dtd', 0.016054167284942168)
('avg_out_avail_max_prev_shop_day', 0.00215132829993096)
('days_til_dept', 0.001577829228033702)
('trailing_avg_solution_counts', 0.0013338665995732376)
('trailing_std_fare', 0.0009053506814738132)
('num_itin_prev_shop_day', 0.0007870914510034137)
('trailing_avg_shop_counts', 0.0007732651422761094)
('dept_dt_dow_int', 0.0007500224969305588)
('fr1_fr2_out_cxrs_overlap', 0.0006952594302412524)
('solution_counts_prev_shop_day', 0.0005848354645387028)
('shop_counts_prev_shop_day', 0.0005764045335395285)
('fr1_fr2_out_cxrs_same', 0.00023070321090940397)
('avg_out_avail_low_prev_shop_day', 0.00011840971159421579)
('is_holiday', 0.0)

Exception in thread cell_monitor-39:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [42]:
plot_feature_impts(rf_feat_imps, 'rf-tuned-fixed-data', "Random Forest - Feature Importances")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-42:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



##

In [ ]:
# rf_cv_model = rf_cv.fit(train_no_hol)
# pred_rf_cv = rf_cv_model.transform(test_no_hol)
# pred_rf_cv = calc_abs_pct_err(pred_rf_cv)

# save errors

In [46]:
pred_rf_cv = pred_rf_cv.withColumn("data_split", F.lit("test"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-46:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [47]:
train_pred_rf_cv = rf_cv_model.transform(train_no_hol)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-47:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [48]:
train_pred_rf_cv = calc_abs_pct_err(train_pred_rf_cv)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-48:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.03701797054042594|
+-------------------+

None

In [49]:
train_pred_rf_cv = train_pred_rf_cv.withColumn("data_split", F.lit("train"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-49:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [52]:
combined_pred_df = pred_rf_cv.union(train_pred_rf_cv)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-52:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [51]:
combined_pred_df.count(), train_pred_rf_cv.count(), pred_rf_cv.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-51:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



(38454, 30790, 7664)

In [53]:
combined_pred_df.coalesce(1).write.parquet("s3://tvlp-ds-users/kendra-frederick/shopping-grid/modeled-data/06b")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-53:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'

